[Fake News](https://www.kaggle.com/shahidkhan1/fake-news-detection-using-lstm-90-accuracy/data)

 **train.csv**: A full training dataset with the following attributes:


*   id: unique id for a news article
*  title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable

>> 1: unreliable

>> 0: reliable

**test.csv**: A testing training dataset with all the same attributes at train.csv without the label.

**submit.csv**


In [1]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/ciclo2/BigData/proyecto-grupal"

In [4]:
!python -m pip install 'fsspec>=0.3.3'

     |████████████████████████████████| 132 kB 29.8 MB/s 


In [2]:
!kaggle competitions download -c fake-news

  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 33.8MB/s]
 53% 5.00M/9.42M [00:00<00:00, 51.5MB/s]
100% 9.42M/9.42M [00:00<00:00, 60.0MB/s]
 89% 33.0M/37.0M [00:00<00:00, 53.7MB/s]
100% 37.0M/37.0M [00:00<00:00, 68.7MB/s]


In [3]:
! unzip test.csv.zip
! unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [6]:
import pandas as pd

In [7]:
df_submit = pd.read_csv("/content/submit.csv")
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")


## Submit

In [8]:
df_submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [9]:
print("Cantidad de elementos SUBMIT:",len(df_submit.index))

Cantidad de elementos SUBMIT: 5200


## Train

In [10]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
print("Cantidad de elementos TRAIN:",len(df_train.index))

Cantidad de elementos TRAIN: 20800


In [12]:
display(type(df_train))
display((df_train.count(), len(df_train.columns)))

pandas.core.frame.DataFrame

(id        20800
 title     20242
 author    18843
 text      20761
 label     20800
 dtype: int64, 5)

In [13]:
df_train = df_train[df_train.text.notna()]

## Test

In [14]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [15]:
print("Cantidad de elementos TEST:",len(df_test.index))

Cantidad de elementos TEST: 5200


In [16]:
display(type(df_test))
display((df_test.count(), len(df_test.columns)))

pandas.core.frame.DataFrame

(id        5200
 title     5078
 author    4697
 text      5193
 dtype: int64, 4)

In [17]:
df_test = df_test[df_test.text.notna()]


## DASK: lecturas

In [18]:
import dask.dataframe as dd

### Submit


In [19]:
%%time
ddf_submit = dd.from_pandas(df_submit, npartitions = 2) 
ddf_submit.compute()

CPU times: user 37.2 ms, sys: 8.79 ms, total: 46 ms
Wall time: 439 ms


### Train

In [20]:
%%time
ddf_train = dd.from_pandas(df_train, npartitions = 2)
ddf_train.compute()


CPU times: user 792 ms, sys: 740 ms, total: 1.53 s
Wall time: 2.21 s


In [21]:
len(ddf_train.index)

20761

### Test

In [22]:
%%time
ddf_test = dd.from_pandas(df_test, npartitions = 2)
ddf_test.compute()

CPU times: user 232 ms, sys: 43.8 ms, total: 276 ms
Wall time: 371 ms


## DASK: Union Test y Submit

In [23]:
ddf_submit.compute()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
...,...,...
5195,25995,0
5196,25996,1
5197,25997,0
5198,25998,1


In [24]:
#type(ddf_submit)
#type(ddf_test_not_null)
#ddf_test_merge_submit = ddf_submit.merge(ddf_test, on="id")
%%time
ddf_test_merge_submit =  ddf_test.merge(ddf_submit, on="id")

CPU times: user 34.3 ms, sys: 578 µs, total: 34.9 ms
Wall time: 34.5 ms


In [25]:
!pip install partd

In [26]:
ddf_test_merge_submit.compute()

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,1
2,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1
3,20811,“Working Class Hero” by John Brennon,Doug Diamond,"Source: CNBC, article by Robert Ferris Arctic ...",1
4,20812,The Rise of Mandatory Vaccinations Means the E...,Shaun Bradley,Written by Shaun Bradley Mandatory vaccinati...,0
...,...,...,...,...,...
2570,25991,E3 Hands-On with ’South Park: The Fractured bu...,Lucas Nolan,The title of South Park: The Fractured but Who...,1
2571,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...,1
2572,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...,0
2573,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...,1


## DASK: remover valores null de train y test

1.   Test



In [27]:
ddf_test_merge_submit['text'].notnull().compute()

0       True
1       True
2       True
3       True
4       True
        ... 
2570    True
2571    True
2572    True
2573    True
2574    True
Name: text, Length: 5193, dtype: bool

In [28]:
%%time
ddf_test_not_null = ddf_test_merge_submit[ddf_test_merge_submit['text'].notnull()].compute() 
ddf_test_not_null

CPU times: user 174 ms, sys: 50.6 ms, total: 225 ms
Wall time: 466 ms


In [29]:
 print("Datos originales:", len(ddf_test), len(ddf_test.columns))
 print("Datos ddf_test_not_null:", len(ddf_test_not_null), len(ddf_test_not_null.columns))

Datos originales: 5193 4
Datos ddf_test_not_null: 5193 5


1.  Train

In [30]:
ddf_train['text'].notnull().compute()

0        True
1        True
2        True
3        True
4        True
         ... 
20795    True
20796    True
20797    True
20798    True
20799    True
Name: text, Length: 20761, dtype: bool

In [31]:
%%time
ddf_train_not_null = ddf_train[ddf_train['text'].notnull()].compute() 
ddf_train_not_null

CPU times: user 402 ms, sys: 222 ms, total: 624 ms
Wall time: 988 ms


In [32]:
 print("Datos originales:", len(ddf_train), len(ddf_train.columns))
 print("Datos ddf_train_not_null:", len(ddf_train_not_null), len(ddf_train_not_null.columns))

Datos originales: 20761 5
Datos ddf_train_not_null: 20761 5


## DASK: pre procesamiento

### Tokenizar

In [33]:
ddf_train_not_null["text"]

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 20761, dtype: object

In [211]:
!pip install dask-ml


     |████████████████████████████████| 147 kB 35.5 MB/s 
     |████████████████████████████████| 802 kB 40.6 MB/s 
     |████████████████████████████████| 15.7 MB 38.8 MB/s 
     |████████████████████████████████| 793 kB 15.2 MB/s 
     |████████████████████████████████| 793 kB 44.8 MB/s 
     |████████████████████████████████| 791 kB 50.8 MB/s 
     |████████████████████████████████| 786 kB 39.4 MB/s 
     |████████████████████████████████| 779 kB 50.5 MB/s 
     |████████████████████████████████| 778 kB 53.7 MB/s 
     |████████████████████████████████| 776 kB 76.6 MB/s 
     |████████████████████████████████| 769 kB 67.8 MB/s 
     |████████████████████████████████| 766 kB 72.9 MB/s 
     |████████████████████████████████| 1.0 MB 42.3 MB/s 
     |████████████████████████████████| 722 kB 45.9 MB/s 
     |████████████████████████████████| 722 kB 57.7 MB/s 
     |████████████████████████████████| 715 kB 55.1 MB/s 
     |████████████████████████████████| 705 kB 52.6 MB/s 
     |███████

In [39]:
from dask_ml.feature_extraction.text import CountVectorizer
import dask.bag as db
from distributed import Client

In [44]:
ddf_train_not_null["text"][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [60]:
tokenizar = ddf_train_not_null["text"][0]
print(tokenizar)

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) 
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. 
As we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emails 

In [62]:
tokenizar.split()

['House',
 'Dem',
 'Aide:',
 'We',
 'Didn’t',
 'Even',
 'See',
 'Comey’s',
 'Letter',
 'Until',
 'Jason',
 'Chaffetz',
 'Tweeted',
 'It',
 'By',
 'Darrell',
 'Lucus',
 'on',
 'October',
 '30,',
 '2016',
 'Subscribe',
 'Jason',
 'Chaffetz',
 'on',
 'the',
 'stump',
 'in',
 'American',
 'Fork,',
 'Utah',
 '(',
 'image',
 'courtesy',
 'Michael',
 'Jolley,',
 'available',
 'under',
 'a',
 'Creative',
 'Commons-BY',
 'license)',
 'With',
 'apologies',
 'to',
 'Keith',
 'Olbermann,',
 'there',
 'is',
 'no',
 'doubt',
 'who',
 'the',
 'Worst',
 'Person',
 'in',
 'The',
 'World',
 'is',
 'this',
 'week–FBI',
 'Director',
 'James',
 'Comey.',
 'But',
 'according',
 'to',
 'a',
 'House',
 'Democratic',
 'aide,',
 'it',
 'looks',
 'like',
 'we',
 'also',
 'know',
 'who',
 'the',
 'second-worst',
 'person',
 'is',
 'as',
 'well.',
 'It',
 'turns',
 'out',
 'that',
 'when',
 'Comey',
 'sent',
 'his',
 'now-infamous',
 'letter',
 'announcing',
 'that',
 'the',
 'FBI',
 'was',
 'looking',
 'into',
 '

In [ ]:
tokenizar_str = tokenizar.astype('str')

In [ ]:
client = Client()
corpus = db.from_sequence(tokenizar, npartitions=2)
corpus
vectorizer = CountVectorizer()
X = vectorizer.get_stop_words(,corpus)


In [57]:
print(corpus)

dask.bag<from_sequence, npartitions=2>


In [35]:
vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '0000',
 '000035',
 '0001',
 '00012',
 '0002',
 '000202',
 '000205',
 '0002062',
 '000billion',
 '000c',
 '000emails',
 '000ft',
 '000k',
 '000km',
 '000lb',
 '000m',
 '000mg',
 '000s',
 '000th',
 '000â',
 '001',
 '0011',
 '0012',
 '0013',
 '002',
 '0020143',
 '0023z',
 '0024',
 '003',
 '004',
 '004s',
 '005',
 '0054z',
 '0057',
 '005s',
 '006',
 '00684',
 '006s',
 '007',
 '007s',
 '008',
 '008s',
 '009',
 '0099',
 '00am',
 '00o',
 '00p',
 '00pm',
 '00μg',
 '01',
 '010',
 '0101',
 '0102',
 '0107y',
 '011s',
 '012',
 '0128',
 '013',
 '0134',
 '0135y',
 '013c2812c9',
 '014',
 '015',
 '016',
 '016e5d9ff252f2444790d05269f4ed90',
 '016s',
 '017',
 '018',
 '019',
 '01915',
 '01am',
 '01pm',
 '01s',
 '01welcome',
 '02',
 '020',
 '0200gmt',
 '021',
 '022',
 '023',
 '0235',
 '024',
 '025',
 '026',
 '027',
 '02714',
 '028',
 '02863',
 '02870',
 '029',
 '02pm',
 '02welcome',
 '03',
 '030',
 '031',
 '032',
 '0325',
 '033',
 '0331',
 '033s',
 '034',
 '035',
 '036',
 '03747',
 '038',